In [10]:
# csv to xml
import csv
import cv2
import numpy as np
import os

In [11]:
direc = 'test'
csv_file_path = '{}_labels.csv'.format(direc)
csv_file = open(csv_file_path, newline='')

In [12]:
spam_reader = csv.reader(csv_file)

In [13]:
properties = []
rows = []
for i, row in enumerate(spam_reader):
    if i == 0:
        properties = row
    elif len(row) != 0:
        rows.append(row)

In [14]:
output_dir = '{}_annotations'.format(direc)

In [15]:
import xml.etree.ElementTree as ET

In [16]:
# create the file structure
def to_annotate(values):
    annot = ET.Element('annotation')
    
    filename = ET.SubElement(annot, 'filename')
    filename.text = values[0]
    
    size = ET.SubElement(annot, 'size')
    width = ET.SubElement(size, 'width')
    width.text = values[1]
    height = ET.SubElement(size, 'height')
    height.text = values[2]
    depth = ET.SubElement(size, 'depth')
    depth.text = '3'
    
    obj = ET.SubElement(annot, 'object')
    name = ET.SubElement(obj, 'name')
    name.text = values[3]
    box = ET.SubElement(obj, 'bndbox')
    xmin = ET.SubElement(box, 'xmin')
    xmin.text = values[4]
    ymin = ET.SubElement(box, 'ymin')
    ymin.text = values[5]
    xmax = ET.SubElement(box, 'xmax')
    xmax.text = values[6]
    ymax = ET.SubElement(box, 'ymax')
    ymax.text = values[7]
    
    
    return annot

In [17]:
prev_file = None
prev_annot = None
for i, values in enumerate(rows):
    cur_annot = to_annotate(values)
    if prev_file is None:
        prev_file = values[0]
        prev_annot = cur_annot
    elif prev_file == values[0]:
        # add box
        obj = ET.Element('object')
        name = ET.SubElement(obj, 'name')
        name.text = values[3]
        box = ET.SubElement(obj, 'bndbox')
        xmin = ET.SubElement(box, 'xmin')
        xmin.text = values[4]
        ymin = ET.SubElement(box, 'ymin')
        ymin.text = values[5]
        xmax = ET.SubElement(box, 'xmax')
        xmax.text = values[6]
        ymax = ET.SubElement(box, 'ymax')
        ymax.text = values[7]
        prev_annot.append(obj)
    else:
        data = ET.tostring(prev_annot).decode('utf-8')
        file_dest = os.path.join(output_dir, prev_file[:-3] + 'xml')
        with open(file_dest, 'w') as f:
            f.write(data)
        prev_file = values[0]
        prev_annot = cur_annot

data = ET.tostring(cur_annot).decode('utf-8')
with open(prev_file[:-3] + 'xml', 'w') as f:
    f.write(data)